In [ ]:
import os
from pyspark.sql import SparkSession

# Configurations

In [ ]:
dbConnectionUrl = "jdbc:postgresql://postgres:5432/dvdrental"
dbProperties = {
    "driver":   "org.postgresql.Driver",
    "user":     "postgres",
    "password": "SuperSecr3t"
}

In [ ]:
minioConfigs = {
    "spark.hadoop.fs.s3a.impl":                "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.endpoint":            "http://minio:9000",
    "spark.hadoop.fs.s3a.access.key":          "admin",
    "spark.hadoop.fs.s3a.secret.key":          "SuperSecr3t",
    "spark.hadoop.fs.s3a.path.style.access":   True
}

In [ ]:
from pyspark import SparkConf

conf = SparkConf()
conf.setAll(minioConfigs.items())
conf.set("spark.jars.packages", "org.postgresql:postgresql:42.2.9")

In [ ]:
spark = SparkSession.builder\
        .appName("Teko::Ingrest")\
        .config(conf=conf)\
        .getOrCreate()

# Read from PostgreSQL

In [ ]:
df = spark.read.jdbc(dbConnectionUrl, "actor", properties=dbProperties)

In [ ]:
df.show(5)

# Write to MinIO

In [ ]:
df.write.parquet("s3a://dwh/actor", mode="overwrite")

# Verify it worked

In [ ]:
df_minio = spark.read.parquet("s3a://dwh/actor")

In [ ]:
df_minio.show(100)